<a href="https://colab.research.google.com/github/van-dang/MRI-Cloud/blob/master/PeriodicDomains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About the code

In [0]:
# This program solves the Bloch-Torrey equation applied to computational diffusion MRI using 
# the finite element method coupled with the theta-method for the spatial discretization.

# The scope of usage: 
# (1) Single domains, Multilayered structures, manifolds
# (2) Membrane permeability for internal interfaces
#     Artificial permeability at the external interfaces
# (3) pure homogeneous Neumann BCs, (4) pseudo-periodic BCs

# Copyright (C) 2019 Van-Dang Nguyen (vdnguyen@kth.se)

# This file is part of DOLFIN.

# DOLFIN is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# DOLFIN is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU Lesser General Public License for more details.

# You should have received a copy of the GNU Lesser General Public License
# along with DOLFIN. If not, see <http://www.gnu.org/licenses/>.

# First added:  2017-10-10
# Last changed: 2019-04-25

# This demo is maintained by Van-Dang Nguyen
# Please report possible problems to vdnguyen@kth.se

# Setting a working environment with FEniCS

In [2]:
# !rm *.py *.xml *.txt *.zip *.pvd *.vtu *.geo *.msh
try:
    from google.colab import files
except:
    print("This is not google.colab")
    
import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
    
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *

gmsh_dir=!which gmsh
if len(gmsh_dir)==0:
    !sudo apt-get update
    !sudo apt-get install -y gmsh
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time

import os, sys, shutil

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

# Disable warnings
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger("rothemain.rothe_utils")
logging.getLogger('UFL').setLevel(logging.WARNING)
logging.getLogger('FFC').setLevel(logging.WARNING)

import warnings
warnings.filterwarnings("ignore")


This is not google.colab


DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


dolfin version: 2019.1.0


#Load pre-defined functions

In [0]:
import sympy as sp  
exists = os.path.isfile('DmriFemBuitInFunc.py')
isupdate = False
if (exists==False or isupdate==True):
    if isupdate==True:
        !rm DmriFemBuitInFunc.py
    print("Load pre-defined functions from GitHub")
    !wget --quiet https://raw.githubusercontent.com/van-dang/MRI-Cloud/master/DmriFemBuitInFunc.py
from DmriFemBuitInFunc import *

# Working on the mesh

In [4]:
def RefineNearBoundary(mesh, nrefine):
    for j in range(nrefine):
      print("Refining ...")
      mesh.init(mesh.topology().dim()-1, mesh.topology().dim()) # Initialise facet to cell connectivity
      markers = MeshFunction("bool", mesh, mesh.topology().dim())
      for c in cells(mesh):
        for f in facets(c):
          if f.exterior():
            markers[c] = True
            break
      mesh = refine(mesh, markers)   
    print("Refined meshes: %d cells, %d vertices"%(mesh.num_cells(), mesh.num_vertices()))
    return mesh


geo_choice = 1;

if geo_choice==1:
    mesh_name = "CirclesInSquare"

    is_partition_function_exist = os.path.isfile('GetPartitionMarkers.py')
    if is_partition_function_exist==False:
        !wget --quiet https://raw.githubusercontent.com/van-dang/MRI-Cloud/mesh/GetPartitionMarkers.py

    is_geo_file_exist = os.path.isfile(mesh_name+'.geo')  
    if is_geo_file_exist==False:
        !wget --quiet https://raw.githubusercontent.com/van-dang/MRI-Cloud/mesh/$mesh_name''.geo

    # Create mesh from geo file by gmsh
    !gmsh -3 $mesh_name''.geo -o $mesh_name''.msh

    # Convert .msh to .xml using dolfin-convert
    !dolfin-convert $mesh_name''.msh $mesh_name''.xml

    clear_output()

    mymesh = Mesh(mesh_name+".xml");  

    from GetPartitionMarkers import *

    GetPartitionMarkers(mesh_name+".msh", "pmk_"+mesh_name+".xml")

    partition_marker = MeshFunction("size_t", mymesh, mymesh.topology().dim())

    File("pmk_"+mesh_name+".xml")>>partition_marker
    
    phase, partion_list = CreatePhaseFunc(mymesh, [], [], partition_marker)
    
    File("Phase.pvd")<<phase
    
elif geo_choice==2:
  
    # clean files
    !rm *msh* *xml* *MAC*
    
    unzip_dir=!which unzip
    if len(unzip_dir)==0:
        !sudo apt-get install -y unzip

    # The periodic meshes were generated with Salome platform
    # ./runAppli -t -u /NOBACKUP/vdnguyen/SalomeMesh/SpheresInBox.py
    # The Salome script is available at
    # https://raw.githubusercontent.com/van-dang/MRI-Cloud/mesh/SpheresInBox.py
    # TODO: The GMSH script for 3D does not work properly for now
    # https://raw.githubusercontent.com/van-dang/MRI-Cloud/mesh/SpheresInBox.geo
    # but it should be fixed in the future
    
    # Download msh files and unzip
    !wget https://github.com/van-dang/MRI-Cloud/raw/mesh/SpheresInBox.msh.zip
    !wget https://github.com/van-dang/MRI-Cloud/raw/mesh/SpheresInBox_cmpt1.msh.zip  
    !unzip -q SpheresInBox.msh.zip
    !unzip -q SpheresInBox_cmpt1.msh.zip 
    
    # clear printscreen
    # clear_output()

    # convert msh to xml
    !dolfin-convert SpheresInBox.msh SpheresInBox.xml
    !dolfin-convert SpheresInBox_cmpt1.msh SpheresInBox_cmpt1.xml
    # create the phase function
    mymesh = Mesh("SpheresInBox.xml");
    cmpt_mesh = Mesh('SpheresInBox_cmpt1.xml')
    mymesh = RefineNearBoundary(mymesh, 1)
    
    evengroup = []
    oddgroup = [cmpt_mesh]
    phase, partion_list, partition_marker = CreatePhaseFunc(mymesh, evengroup, oddgroup, None)                

    File("Phase.pvd")<<phase
    
CheckAndCorrectPeriodicity(mymesh, 0, 1e-6)
CheckAndCorrectPeriodicity(mymesh, 1, 1e-6)
CheckAndCorrectPeriodicity(mymesh, 2, 1e-6)

Extracting cell markers from: CirclesInSquare.msh ...
Extracted successfully on: 6610 elements
Partition marker list: ['1', '0']
Wrote to: pmk_CirclesInSquare.xml
Check and correct the periodicity in direction 0
  The maximum error in y-component is: 1.33226762955e-15
  The maximum error in z-component is: 0
  The mesh was successfully corrected for this direction.
Check and correct the periodicity in direction 1
  The maximum error in x-component is: 1.33226762955e-15
  The maximum error in z-component is: 0
  The mesh was successfully corrected for this direction.
Check and correct the periodicity in direction 2
  Direction 2 is invalid


# Solve the Bloch-Torrey equation

In [0]:
mri_simu = MRI_simulation()
mri_para = MRI_parameters()

# bvalues = [0, 500, 1000, 2000, 3000, 4000, 6000, 10000]
bvalues = [500, 1000, 2000, 3000, 4000, 6000, 10000]
for bvalue in bvalues:
    #################################################################################
    #########################  Pre-defined parameters ###############################
    mri_para.stype='PGSE'                                # sequence type
    mri_para.bvalue = bvalue;                            # bvalue
    mri_para.delta, mri_para.Delta = 10000, 13000           # time sequence
    mri_para.set_gradient_dir(mymesh, 1, 1, 0)           # gradient direction

    mri_para.T = mri_para.Delta+mri_para.delta
    mri_para.fs_sym = sp.Piecewise(
                    (  1., mri_para.s < mri_para.delta ),
                    (  0., mri_para.s < mri_para.Delta ),
                    ( -1., mri_para.s < mri_para.T ),
                    (  0., True )  
                ) 

    mri_para.Apply()
    mri_simu.k = 10;                                    # time-step size
    mri_simu.nskip = 5;                                  # frequency to print ouputs
    mydomain = MyDomain(mymesh, mri_para)
    mydomain.phase = phase 
    mydomain.PeriodicDir = [1, 1, 0];             # Direction of the periodicity
    mydomain.IsDomainPeriodic = False            # Confirm if the mesh if periodic
    mydomain.IsDomainMultiple = True              # Confirm if the mesh is multiple
    ################################################################################
    # mydomain.kappa_e_scalar = 5e-4;
    mydomain.kappa = 1e-5;
    mydomain.Apply()   # Call Apply before setting the diffusion tensor
    ################################################################################
    # Impose the diffusion coefficient
    mydomain.D  = 3e-3;
    #################################################################################

    # linsolver = PETScLUSolver("mumps")
    linsolver = KrylovSolver("bicgstab")
    linsolver.parameters["absolute_tolerance"] = 1e-4
    linsolver.parameters["relative_tolerance"] = 1e-4
    linsolver.parameters["maximum_iterations"] = 10000
    
    mri_simu.solve(mydomain, mri_para, linsolver)

    ctext = ""
    if (bvalue==0):
        ctext = "# weak-periodic-kappa1e_5-dt_10"
    Post_processing(mydomain, mri_para, mri_simu, plt, ctext)    

Domain size: xmin=-5.000000, ymin=-5.000000, zmin=0.000000, xmax=5.000000, ymax=5.000000, zmax=0.000000
Function Space for Two-compartment Domains has 4 components
(ur0, ui0, ur1, ur1): r-real, i-imaginary
Initialize a standard function space.
The pseudo-periodic BCS are weakly imposed.
The mesh does not need to be periodic.
t:   0.00  T: 23000.00 dt: 10.0 qvalue: 2.274294e-05 Completed 0.00%
t:  50.00  T: 23000.00 dt: 10.0 qvalue: 2.274294e-05 Completed 0.22%
t: 100.00  T: 23000.00 dt: 10.0 qvalue: 2.274294e-05 Completed 0.43%
t: 150.00  T: 23000.00 dt: 10.0 qvalue: 2.274294e-05 Completed 0.65%
t: 200.00  T: 23000.00 dt: 10.0 qvalue: 2.274294e-05 Completed 0.87%
t: 250.00  T: 23000.00 dt: 10.0 qvalue: 2.274294e-05 Completed 1.09%
t: 300.00  T: 23000.00 dt: 10.0 qvalue: 2.274294e-05 Completed 1.30%
t: 350.00  T: 23000.00 dt: 10.0 qvalue: 2.274294e-05 Completed 1.52%
t: 400.00  T: 23000.00 dt: 10.0 qvalue: 2.274294e-05 Completed 1.74%
t: 450.00  T: 23000.00 dt: 10.0 qvalue: 2.274294e-05